# Chains in LangChain
## Outline
- LLMChain
  - Basic chain type
- Sequential Chains
  - Sequential chain is another type of chains.
    The idea is to combine multiple chains where the output of the one chain is the input of the next chain.
  - There is two type of sequential chains:
    1. SimpleSequentialChain: Single input/output
    2. SequentialChain: multiple inputs/outputs
- Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!pip install pandas

In [3]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [4]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [5]:
from models import get_chat_model

llm = get_chat_model(temperature=0.9)

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [7]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt) # RunnableSequence

/var/folders/l4/vd99d1jj46l4pg5l2_852z280000gn/T/ipykernel_26058/1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
product = "Queen Size Sheet Set"
chain.run(product) # invoke()

/var/folders/l4/vd99d1jj46l4pg5l2_852z280000gn/T/ipykernel_26058/550859008.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'Here are some name ideas, broken down by category, for a company that makes Queen Size Sheet Sets:\n\n**Focusing on Quality & Comfort:**\n\n*   **Queen Comfort Linens:** Simple, direct, and emphasizes comfort.\n*   **Silken Queen:** Evokes luxury and softness.\n*   **DreamWeave Queen:** Highlights the weaving process and a dreamy sleep.\n*   **Queen\'s Embrace:** Implies a comforting and enveloping feel.\n*   **Royal Rest:** Suggests high quality and restful sleep.\n*   **Queenly Slumber:** Focuses on the quality of sleep.\n*   **CloudSoft Queen:** Highlights the soft texture.\n*   **The Queen\'s Thread:** Suggests craftsmanship and quality.\n\n**Focusing on Simplicity & Essentials:**\n\n*   **Queen Sheet Co.** (or Queen Sheet Company): Straightforward and easy to remember.\n*   **Queen Size Basics:** Highlights the core product.\n*   **Simple Sheets Queen:** Emphasizes simplicity and ease of use.\n*   **The Queen\'s Standard:** Implies reliability and a benchmark of quality.\n*   **Q

## SimpleSequentialChain

In [14]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}? Provide single suggestion"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
from langchain.chains import SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
QueenSheets
QueenSheets: Luxurious, high-quality sheets designed for ultimate comfort and a regal night's sleep. Experience the difference, fit for royalty.

> Finished chain.


"QueenSheets: Luxurious, high-quality sheets designed for ultimate comfort and a regal night's sleep. Experience the difference, fit for royalty."

## SequentialChain

![Sequential Chain](images/sequential_chain.png)

In [18]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [19]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [20]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [24]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a single variant of follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [25]:
from langchain.chains import SequentialChain

# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [26]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': "This reviewer believes the product's taste is subpar and the foam quality is poor compared to store-bought versions, suggesting a potential issue with the batch or authenticity.",
 'followup_message': 'Okay, here\'s a possible follow-up response in French, addressing the concerns about taste and foam quality:\n\n**Response:**\n\n"Bonjour, nous sommes désolés d\'apprendre que vous n\'avez pas été satisfait du goût et de la qualité de la mousse de notre produit. Nous prenons vos commentaires très au sérieux. Il est possible qu\'il y ait eu un problème avec ce lot spécifique, ou que le produit ait été affecté durant le transport. 

## Router Chain

![Router Chain](images/router_chain.png)

In [27]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [28]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [29]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [30]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [31]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [32]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [33]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [34]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/l4/vd99d1jj46l4pg5l2_852z280000gn/T/ipykernel_26058/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [35]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Ah, a classic! Black body radiation is essentially the electromagnetic radiation emitted by an object that absorbs all incident electromagnetic radiation, regardless of frequency or angle. Think of it like this:\n\n*   **Perfect Absorber:** A black body doesn't reflect or transmit any light; it absorbs everything that hits it.\n\n*   **Re-emits Energy:** Because it absorbs energy, it also has to re-emit that energy as electromagnetic radiation.  The spectrum and intensity of this emitted radiation depends *only* on the object's temperature.\n\n*   **Characteristic Spectrum:** The emitted radiation has a specific, continuous spectrum called the black-body spectrum. This spectrum peaks at a certain wavelength that is inversely proportional to the temperature (Wien's displacement law).  Also, the total power radiated is proportional to the fourth power of the temperature (Stefan-Boltzmann law).\n\nSo, in short, black body radiation is the thermal electromagnetic radiation emitted by a th

In [36]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2?'}
> Finished chain.


"Okay, this is a classic! Let's break it down, even though it's quite simple:\n\n*   **Identify the components:** We have two numbers, 2 and 2, and an addition operation (+).\n\n*   **Perform the operation:** Addition means combining the quantities. So, we're combining 2 units with another 2 units.\n\n*   **Determine the result:** If you have two things and add two more, you end up with four things.\n\nTherefore, 2 + 2 = 4."

In [37]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the blueprint and instruction manual for building and operating a human being. Here's a breakdown of why it's essential in every cell:\n\n*   **Genetic Information:** DNA contains all the genetic information necessary to create and maintain an organism. It holds the instructions for building proteins, which are the workhorses of the cell, carrying out a vast range of functions from catalyzing chemical reactions to forming structural components.\n*   **Cellular Function:** DNA dictates what kind of cell it will be (e.g., muscle cell, nerve cell, skin cell) and what functions it will perform. Different cell types express different genes (segments of DNA) to carry out their specific roles.\n*   **Replication and Inheritance:** DNA is self-replicating. When a cell divides, its DNA is duplicated, ensuring that each daughter cell receives a complete and accurate copy of the genetic information. This is essential for growth, repair, and repr